In [0]:
#define parameters
INP_SRC_SYS="testmax"
SRC_STREAM_ID="TESTMAX001"
PARTITION_DATE="FROM_FILE"
WRITE_MODE="Overwrite"
#ENV="dev"
DBFS_URL="dbfs:/"          #+ENV
import datetime
date_time=datetime.datetime.now().strftime("%Y%m%d%H%M%S")
CREATE_DATE= datetime.datetime.now().strftime("%Y-%m-%d")
print(CREATE_DATE)
current_year=CREATE_DATE.split("-")[0].lstrip('0')
current_month=CREATE_DATE.split("-")[1].lstrip('0')
current_day=CREATE_DATE.split("-")[2].lstrip('0')
BATCH_ID = SRC_STREAM_ID+"_"+date_time
print(BATCH_ID)
print(date_time)

2022-03-25
TESTMAX001_20220325081304
20220325081304


In [0]:
import os
import re
def db_list_files(file_path, file_prefix):
  #file_list = [file.path for file in dbutils.fs.ls(file_path) if os.path.basename(file.path).startswith(file_prefix)]
  file_list = [file.path for file in dbutils.fs.ls(file_path) if re.search(file_prefix, os.path.basename(file.path))]
  return file_list
#db_list_files(DBFS_URL+"/tmp/data/payment/bgmax/", "^bgmax_bettdfi_[0-9]{8}_[0-9]")

In [0]:
def meta_model(INP_SRC_SYS):
  try:
    metamodel_full_df = (spark.read           # The DataFrameReader
       .option("sep", "|")        # Use tab delimiter (default is comma-separator)
      #.option("header", "true")   # Use first line of all files as header
       .schema(METAMODEL_SCHEMA)   #Apply schema
       .csv(METAMODEL_LOOKUP)               # Creates a DataFrame from CSV after reading in the file
    )
    #metamodel_full_df.show(vertical=True)
    metamodel_df=metamodel_full_df.filter(metamodel_full_df.SRC_STREAM_ID==SRC_STREAM_ID)
    #metamodel_df.show(vertical=True)
    return metamodel_df
  except:
    print("Metamodel doesnt exists for this source")
#metamodel_df.show()

In [0]:
def file_exists(path):
  try:
    dbutils.fs.ls(path)
    #print("The path ",path," exists")
    pass
    return True
  except:
    print("The path ",path," does not exist")
    return False

In [0]:
def create_partions_from_file(input_dir,part_keys):
  #part_keys="ing_year=YYYY:ing_month=MM:ing_day=DD"
  partition_values = {}
  partitons=part_keys.split(":")
  #len=len(xx)
  partition_k = input_dir
  for i in partitons:
    #print(i)
    if i == 'ing_year=YYYY' :
      partition_k = partition_k + '/ing_year='+current_year
      partition_values['ing_year']=current_year
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_month=MM':
      partition_k = partition_k + '/ing_month='+current_month
      partition_values['ing_month']=current_month
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_day=DD':
      partition_k = partition_k + '/ing_day='+current_day
      partition_values['ing_day']=current_day
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'create_dir':
      print("Creating input_dir", partition_k)
      dbutils.fs.mkdirs(partition_k)
      return_value = partition_k
  #print("partition_k",partition_k)
  #print("partition_values",partition_values)
  return return_value

In [0]:
def create_partions_with_current_date(input_dir,part_keys):
  #part_keys="ing_year=YYYY:ing_month=MM:ing_day=DD"
  partition_values = {}
  partitons=part_keys.split(":")
  #len=len(xx)
  partition_k = input_dir
  for i in partitons:
    #print(i)
    if i == 'ing_year=YYYY' :
      partition_k = partition_k + '/ing_year='+current_year
      partition_values['ing_year']=current_year
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_month=MM':
      partition_k = partition_k + '/ing_month='+current_month
      partition_values['ing_month']=current_month
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'ing_day=DD':
      partition_k = partition_k + '/ing_day='+current_day
      partition_values['ing_day']=current_day
      return_value = partition_values
      #if not file_exists(partition_k):
       # print("Creating directory ",partition_k,"now")
        #dbutils.fs.mkdirs(partition_k)
    elif i == 'create_dir':
      print("Creating input_dir", partition_k)
      dbutils.fs.mkdirs(partition_k)
      return_value = partition_k
  #print("partition_k",partition_k)
  #print("partition_values",partition_values)
  return return_value

In [0]:
def get_metamodel_schema():
  METAMODEL_SCHEMA = StructType([ \
    StructField("PROCESS_ID",StringType(),True), \
    StructField("PROCESS_TYPE",StringType(),True), \
    StructField("SRC_ID",StringType(),True), \
    StructField("SRC_SYS_NM",StringType(),True), \
    StructField("STREAM_NM",StringType(),True), \
    StructField("SRC_STREAM_ID",StringType(),True), \
    StructField("SRC_SYS_TYPE",StringType(),True), \
    StructField("SRC_SYS_DESC",StringType(),True), \
    StructField("FIELD_SEP",StringType(),True), \
    StructField("LINE_SEP",StringType(),True), \
    StructField("HEADER_ROW",StringType(),True), \
    StructField("FOOTER_ROW",StringType(),True), \
    StructField("LOAD_TYPE",StringType(),True), \
    StructField("WHERE_CLUASE",StringType(),True), \
    StructField("PARTITION_KEY",StringType(),True), \
    StructField("CUST_DATA_FLG",StringType(),True), \
    StructField("CUST_DATA_DESC",StringType(),True), \
    StructField("LEGAL_GRAND",StringType(),True), \
    StructField("LOAD_FREQ",StringType(),True), \
    StructField("LANDING_DIR",StringType(),True), \
    StructField("RAW_ODL_LOC",StringType(),True), \
    StructField("BASE_TBL_NAME",StringType(),True), \
    StructField("BASE_ODL_LOC",StringType(),True), \
    StructField("SCHEMA_DIR",StringType(),True), \
    StructField("C1_TBL_NAME",StringType(),True), \
    StructField("C1_ODL_LOC",StringType(),True), \
    StructField("DELIVERY_FORMAT",StringType(),True), \
    StructField("SRV_NAME",StringType(),True), \
    StructField("SRV_SHRT_NAME",StringType(),True), \
    StructField("SRC_TIME_ZONE",StringType(),True) \
  ])
  return METAMODEL_SCHEMA

In [0]:
#define Dropzone to Raw pipeline
def dropzone_to_raw(INP_DF):
  print("###########################DropZone to RAW Pipeline ##################################")
  #INP_DF.show(vertical=True)
  #input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()
  input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()[0]
  LANDING_DIR=INP_DF.select("LANDING_DIR").rdd.flatMap(list).collect()[0]
  #dbfs_land_dir=LANDING_DIR.replace("/mnt/shareddisk",DBFS_URL+"/tmp")
  dbfs_land_dir=LANDING_DIR
  print("dbfs_land_dir:",dbfs_land_dir)
  RAW_ODL_LOC=INP_DF.select("RAW_ODL_LOC").rdd.flatMap(list).collect()[0]
  print("RAW_ODL_LOC:",RAW_ODL_LOC)
  dbfs_raw_file=DBFS_URL+"odl1/raw" + RAW_ODL_LOC + CREATE_DATE.replace("-",'')
  
  print("input file name is :",input_file)
  print("Landing directory is :",LANDING_DIR)
  print("Raw directory is :",RAW_ODL_LOC)
  print("Databricks Raw Loc is :",dbfs_raw_file)
  #File listing
  #print("File listing",dbutils.fs.ls(dbfs_land_dir))
  landing_file = db_list_files(dbfs_land_dir, input_file)[0]
  raw_file_name = landing_file.split(os.sep)[-1]
  #check if directory exists, else create it
  if not file_exists(dbfs_raw_file):
    print("Creating directory ",dbfs_raw_file,"now")
    dbutils.fs.mkdirs(dbfs_raw_file)
  dbfs_raw_file = dbfs_raw_file +"/"+ raw_file_name
  print("RAW FILE:",dbfs_raw_file)
  if not file_exists(dbfs_raw_file):
  # Copy the file from Landing directory to Raw folder  
    print("Copying the file ",landing_file,"to Raw ADLS ",dbfs_raw_file)
    dbutils.fs.cp(landing_file,dbfs_raw_file)
  print("###########################DropZone to RAW Pipeline Completed Successfully##################################")
    
  return dbfs_raw_file

In [0]:
#define Raw to Base pipeline
from pyspark.sql.functions import lit
def raw_to_base_csv(INP_DF,INP_FILE):
  print("###########################RAW to BASE Pipeline ##################################")
  #INP_DF.show(vertical=True)
  #input_file=INP_DF.select("STREAM_NM").rdd.flatMap(list).collect()
  BASE_ODL_LOC=INP_DF.select("BASE_ODL_LOC").rdd.flatMap(list).collect()[0]
  BASE_TBL_NAME = INP_DF.select("BASE_TBL_NAME").rdd.flatMap(list).collect()[0]
  PARTITION_KEY = INP_DF.select("PARTITION_KEY").rdd.flatMap(list).collect()[0]
  FIELD_SEP = INP_DF.select("FIELD_SEP").rdd.flatMap(list).collect()[0]
  if FIELD_SEP == 'PIPE':    FIELD_SEP = '|'
  else: FIELD_SEP
  dbfs_base_dir=DBFS_URL+"odl1/base" + BASE_ODL_LOC + BASE_TBL_NAME.split("_")[1]
  
  #Create partition directories
  if PARTITION_DATE=='FROM_FILE':
    base_dir=create_partions_from_file(dbfs_base_dir,"create_dir")
    #get partition values
    partition_values=create_partions_from_file(dbfs_base_dir,PARTITION_KEY)
  else:
    base_dir=create_partions_with_current_date(dbfs_base_dir,"create_dir")
    #get partition values
    partition_values=create_partions_from_file(dbfs_base_dir,PARTITION_KEY)

  
  print("BASE_TBL_NAME  is :",BASE_TBL_NAME)
  print("BASE_ODL_LOC  is :",dbfs_base_dir)
  print("PARTITION_KEY  is :",PARTITION_KEY)
  print("Base Dir is :",base_dir)
  print("INP_FILE is :",INP_FILE)
  print("FIELD_SEP is :",FIELD_SEP)
    
  
  #Read Input file
  raw_df = (spark.read
            .option("sep",FIELD_SEP)
            .option("header",True)
            .csv(INP_FILE)
            )
  #raw_df.show()
  repartitionedDF = raw_df.repartition(4)
  enrich_df=repartitionedDF.select(lit(SRC_STREAM_ID).alias("src_stream_id"),
                          lit(BATCH_ID).alias("batch_id"),
                          lit(CREATE_DATE).alias("create_date"),
                          lit("UTC").alias("time_zone"),
                          lit(CREATE_DATE).alias("src_business_date"),
                          "*")
  #enrich_df.show()
  print("partition_values:",partition_values)
  parts=""
  for k in partition_values:
  #  print(k,d[k])
  #  print(k)
    parts=parts+"/"+k+"="+partition_values[k]
    enrich_df=enrich_df.withColumn(k,lit(partition_values[k]))
  output_schema=enrich_df.schema
  output_file=base_dir+parts
  print("Writing file ", output_file, "with ",enrich_df.rdd.getNumPartitions()," partitions")
  enrich_df.write.format("avro").partitionBy(list(partition_values.keys())).mode(WRITE_MODE).save(base_dir)
  print("###########################RAW to BASE Pipeline Completed Succefully ##################################")
 
  return output_file


In [0]:
# Metamodel lookup reading
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
#import sys
#defining metamodel Schema
METAMODEL_SCHEMA=get_metamodel_schema()
try:
  METAMODEL_LOOKUP = DBFS_URL+"mnt/shareddisk/data/lookup/"+INP_SRC_SYS+"_meta_config.lkp"
  #Read lookup files
  #Call Metamodel to fetch data from metamodel lookup
  metamodel_df=meta_model(INP_SRC_SYS)
  #metamodel_df.show(vertical=True)
  metamodel_src2dropzone_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="SOURCE_TO_DROPZONE")
  metamodel_dropzone2raw_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="DROPZONE_TO_RAW")
  metamodel_raw2base_df=metamodel_df.filter(metamodel_df.PROCESS_TYPE=="RAW_TO_BASE")
  #Dropzone to Raw
  dbfs_raw_file = dropzone_to_raw(metamodel_dropzone2raw_df)
  #Raw to Base
  BASE_ODL_LOC=metamodel_raw2base_df.select("BASE_ODL_LOC").rdd.flatMap(list).collect()[0]
  BASE_TBL_NAME = metamodel_raw2base_df.select("BASE_TBL_NAME").rdd.flatMap(list).collect()[0]
  dbfs_base_dir=DBFS_URL+"odl1/base" + BASE_ODL_LOC + BASE_TBL_NAME.split("_")[1]
  output_file=raw_to_base_csv(metamodel_raw2base_df,dbfs_raw_file)
  #Move raw file to Processed Folder
  
  #metamodel_raw2base_df.show(vertical=True)

except:
  import sys
  print("Metamodel lookup file",METAMODEL_LOOKUP," doesnt exists for this source",sys.exc_info()[0], "occurred.")
  #print("Metamodel lookup file",METAMODEL_LOOKUP," doesnt exists for this source")
        #ys.exc_info()[0], "occurred.")



###########################DropZone to RAW Pipeline ##################################
dbfs_land_dir: /mnt/shareddisk/data/test/testmax/
RAW_ODL_LOC: /data/test/testmax/tstdfi/
input file name is : ^testmax_tstdfi_[0-9]{8}_[0-9]
Landing directory is : /mnt/shareddisk/data/test/testmax/
Raw directory is : /data/test/testmax/tstdfi/
Databricks Raw Loc is : dbfs:/odl1/raw/data/test/testmax/tstdfi/20220325
RAW FILE: dbfs:/odl1/raw/data/test/testmax/tstdfi/20220325/testmax_tstdfi_20220301_20220301154848
###########################DropZone to RAW Pipeline Completed Successfully##################################
###########################RAW to BASE Pipeline ##################################
Creating input_dir dbfs:/odl1/base/data/test/testmax/tstdfi
BASE_TBL_NAME  is : testmax_tstdfi
BASE_ODL_LOC  is : dbfs:/odl1/base/data/test/testmax/tstdfi
PARTITION_KEY  is : ing_year=YYYY:ing_month=MM:ing_day=DD
Base Dir is : dbfs:/odl1/base/data/test/testmax/tstdfi
INP_FILE is : dbfs:/odl1/raw/data/te

In [0]:
dbutils.fs.ls(output_file)

Out[11]: [FileInfo(path='dbfs:/odl1/base/data/test/testmax/tstdfi/ing_year=2022/ing_month=3/ing_day=25/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1648196013000),
 FileInfo(path='dbfs:/odl1/base/data/test/testmax/tstdfi/ing_year=2022/ing_month=3/ing_day=25/_committed_2675974476602157253', name='_committed_2675974476602157253', size=380, modificationTime=1648196012000),
 FileInfo(path='dbfs:/odl1/base/data/test/testmax/tstdfi/ing_year=2022/ing_month=3/ing_day=25/_started_2675974476602157253', name='_started_2675974476602157253', size=0, modificationTime=1648196010000),
 FileInfo(path='dbfs:/odl1/base/data/test/testmax/tstdfi/ing_year=2022/ing_month=3/ing_day=25/part-00000-tid-2675974476602157253-b928a120-b8e3-4c8f-8396-e084ff02a712-12-1.c000.avro', name='part-00000-tid-2675974476602157253-b928a120-b8e3-4c8f-8396-e084ff02a712-12-1.c000.avro', size=997, modificationTime=1648196011000),
 FileInfo(path='dbfs:/odl1/base/data/test/testmax/tstdfi/ing_year=2022/ing_month=3/ing_day=25/p

In [0]:
#Verify output file
print(dbfs_base_dir)
base_df = spark.read.format("avro").load(dbfs_base_dir)
base_df.show()          
             

dbfs:/odl1/base/data/test/testmax/tstdfi
+-------------+--------------------+-----------+---------+-----------------+-------+--------------------+------------+--------------------+------------+----------------------+---------------------+----------------------+--------+---------+-------+
|src_stream_id|            batch_id|create_date|time_zone|src_business_date|FILE_ID|         LAYOUT_NAME|FILE_VERSION|           TIMESTAMP|CREDIT_ROUND|NUM_OF_PAYMENT_RECORDS|NUM_OF_DEDUCT_RECORDS|NUM_OF_DEPOSIT_RECORDS|ing_year|ing_month|ing_day|
+-------------+--------------------+-----------+---------+-----------------+-------+--------------------+------------+--------------------+------------+----------------------+---------------------+----------------------+--------+---------+-------+
|   TESTMAX001|TESTMAX001_202203...| 2022-03-25|      UTC|       2022-03-25|     61|TESTNSUPP FIL    ...|          01|2016-04-28-08.45....|       TST56|                     1|                    1|                  